In [12]:
from astropy.io import fits
import numpy as np
import glob
import os
import pathlib
import re
from datetime import datetime

In [6]:
hdulist = fits.open("../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20050929/st_2418807291_p.fit")  #itokawa
hdu = hdulist[0]
data = hdu.data
header = hdu.header
date = header["UTC_0"]
date = re.split('[-T]',date)

date

['2005', '09', '29', '15:10:23.']

In [13]:
hdulist = fits.open("../../gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/20040517/st_1035463706_p.fit") #moon
hdu = hdulist[0]
data = hdu.data
header = hdu.header
filt = header["FILTER_0"]
startH = header["START_H"]
startV = header["START_V"]
x_axis = header["NAXIS1"]
y_axis = header["NAXIS2"]
date = header["UTC_0"]
date = re.split('[-T]',date)

In [14]:
l_year = 2003
l_month = 5
l_day = 9
l_date = datetime(l_year,l_month,l_day)

year = int(date[0])
month = int(date[1])
day = int(date[2])
date = datetime(year,month,day)
days = date - l_date
DAY = days.days

In [15]:
def get_bias(DAY):
    result = 0

    B0 = 3.18 * pow(10,2)
    B1 = -4.12 * pow(10,-2)
    B2 = 2.00 * pow(10,-5)

    result = B0 + (B1 * DAY) + (B2 * pow(DAY,2))

    return result

def get_flat(filt):
    result = 0
    home_dir = os.path.expanduser("~")
    p = os.path.join(home_dir, "Documents/gra/HAY_A_AMICA_3_HAYAMICA_V1_0/data/preflight")
    for f in glob.glob(p+"/flat_"+filt+".fit"):
        result = fits.open(f)
        result = result[0].data
    return result
def flat_cor(filt, data,startH,startV,x_axs,y_axs):
    result = 0
    flat_im = get_flat(filt)
    
    result = data/flat_im[startV:startV+y_axs,startH:startH+x_axs]

    return result

In [16]:
bias = get_bias(DAY)
moon_bias = data - bias
result = flat_cor(filt, moon_bias,startH,startV,x_axis,y_axis)
hdu = fits.PrimaryHDU(result)
hdulist = fits.HDUList([hdu])
hdulist.writeto('flatbias_moon.fits',overwrite=True)

In [17]:
# サンプルの配列を作成します（例としてランダムな数値を使用）
array = moon_bias

# 条件を満たす要素のみを選択します
filtered_array = array[array >= 500]

# 条件を満たす要素の平均を計算します
average_above_1000 = np.mean(filtered_array)

print("1000以上の値を持つ要素の平均:", average_above_1000)

1000以上の値を持つ要素の平均: 819.7576625873696


In [ ]:
data = fits.getdata("../HAY_A_AMICA_3_HAYAMICA_V1_0/data/20051017/st_2468186849_p.fit")
